In [1]:
import numpy as np
import pandas as pd

In [2]:
df_merged = pd.read_csv("../data/df_merged.csv")

In [14]:
# randomnisation des utilisateurs group A/B pour un test A/B

np.random.seed(42)

users = df_merged['visitorid'].unique()

ab_groups = pd.DataFrame({
    'visitorid': users,
    'group': np.random.choice(['A', 'B'], size=len(users))
})

df_ab = df_merged.merge(ab_groups, on='visitorid', how='left')
df_ab['group'].value_counts()



group
B    2954777
A    2945486
Name: count, dtype: int64

In [15]:
user_purchase = (
    df_ab[df_ab['event'] == 'transaction']
    .groupby(['visitorid', 'group'])
    .size()
    .reset_index(name='nb_transactions')
)

user_purchase['purchased'] = 1

print(user_purchase.head())


   visitorid group  nb_transactions  purchased
0        172     B                2          1
1        186     B                6          1
2        264     B                2          1
3        419     B                1          1
4        539     B                1          1


In [16]:
# Tous les utilisateurs (acheteurs + non-acheteurs)

users_all = ab_groups.merge(
    user_purchase[['visitorid', 'group', 'purchased']],
    on=['visitorid', 'group'],
    how='left'
)

users_all['purchased'] = users_all['purchased'].fillna(0)

print(users_all.head())


   visitorid group  purchased
0     257597     A        0.0
1     992329     B        0.0
2     111016     A        0.0
3     483717     A        0.0
4     951259     A        0.0


In [ ]:
# Calcul du KPI par groupe

In [10]:
kpi = users_all.groupby('group')['purchased'].agg(
    conversions='sum',
    visitors='count'
)

kpi['conversion_rate'] = kpi['conversions'] / kpi['visitors']
kpi


,conversions,visitors,conversion_rate
group,,,
A,5884.0,704323,0.008354
B,5835.0,703257,0.008297


nous constatons donc a la suite de notre simulation que la Proportion de visiteurs ayant effectué au moins une transaction du groupe A et legerement superieure  à celle du groupe B avec un avantage du groupe A de 49 d'achats


In [ ]:
# / TEST STATISTIQUE : Test de proportion

In [17]:
# Calcule de  la proportion combinée (pooled)

p_pool = kpi['conversions'].sum() / kpi['visitors'].sum()
print(f"Proportion combinée (pooled): {p_pool:.4f}")

Proportion combinée (pooled): 0.0083


In [18]:
# Calcule du Z-score
p_A = kpi.loc['A', 'conversion_rate']
p_B = kpi.loc['B', 'conversion_rate']   
n_A = kpi.loc['A', 'visitors']
n_B = kpi.loc['B', 'visitors']
std_error = np.sqrt(p_pool * (1 - p_pool) * (1/n_A + 1/n_B))    
z_score = (p_B - p_A) / std_error   
print(f"Z-score: {z_score:.4f}")        

Z-score: -0.3722


In [19]:
# Calcule du p-value

from scipy.stats import norm
p_value = 2 * (1 - norm.cdf(abs(z_score)))
print(f"P-value: {p_value:.4f}")

P-value: 0.7097


Avec une p-value de 0,7 (> 0,05), nous ne rejetons pas l’hypothèse nulle. Il n’existe pas de différence statistiquement significative entre les groupes.
meme si A reste superieur à B

In [ ]:
# Analyse Business

Le test A/B ne met en évidence aucune différence statistiquement significative entre les variantes A et B (p-value = 0,7). La légère variation observée est négligeable et attribuable au hasard. La variante B ne justifie pas un déploiement en production. Il est recommandé de conserver la version A et de concevoir une nouvelle variante avec un impact potentiel plus fort.